### [207\. Course Schedule](https://leetcode.com/problems/course-schedule/)

Difficulty: **Medium**  

Related Topics: [Depth-first Search](https://leetcode.com/tag/depth-first-search/), [Breadth-first Search](https://leetcode.com/tag/breadth-first-search/), [Graph](https://leetcode.com/tag/graph/), [Topological Sort](https://leetcode.com/tag/topological-sort/)


There are a total of `numCourses` courses you have to take, labeled from `0` to `numCourses-1`.

Some courses may have prerequisites, for example to take course 0 you have to first take course 1, which is expressed as a pair: `[0,1]`

Given the total number of courses and a list of prerequisite **pairs**, is it possible for you to finish all courses?

**Example 1:**

```
Input: numCourses = 2, prerequisites = [[1,0]]
Output: true
Explanation: There are a total of 2 courses to take. 
             To take course 1 you should have finished course 0\. So it is possible.
```

**Example 2:**

```
Input: numCourses = 2, prerequisites = [[1,0],[0,1]]
Output: false
Explanation: There are a total of 2 courses to take. 
             To take course 1 you should have finished course 0, and to take course 0 you should
             also have finished course 1\. So it is impossible.
```

**Constraints:**

*   The input prerequisites is a graph represented by **a list of edges**, not adjacency matrices. Read more about .
*   You may assume that there are no duplicate edges in the input prerequisites.
*   `1 <= numCourses <= 10^5`

### Cycle Detection by DFS

[參考影片](https://youtu.be/rKQaZuoUR4M)

類似 287. Find the Duplicate Number

dfs 遍歷 prerequisites 如果有 cycle 代表無法完成

使用三個 set 將每個節點分類

1. white set: 尚未 visit
2. grey set: 已經 visit, 但還有 neighbor 還沒 visit
3. black set: 所有 neighbor 都被 visit

檢查 neighbor 的狀態:

1. 該 neighbor 在 white set: 還沒 visit 過, 放進 grey set
2. 該 neighbor 在 grey set: 發現 cycle 回傳 False
3. 該 neighbor 在 black set: 已經 visit 過了, 不需要再次遍歷

time complexity: $O(V^2 + E)$

In [ ]:
from collections import defaultdict
class Solution:
    def canFinish(self, numCourses: int, prerequisites: List[List[int]]) -> bool:
        # make graph
        white = defaultdict(list)
        for x, y in prerequisites:
            white[y].append(x)
        sorted(white.items())
        
        grey = []
        black = []
        
        def dfs(i):
            if i in black:
                return True # no need to visit
            if i in grey:
                return False # with cycle
            else:
                grey.append(i)
                for j in white[i]:
                    if not dfs(j): # with cycle
                        return False # with cycle
                black.append(i)
                return True # no cycle
        
        for i in range(numCourses):
            if not dfs(i): # with cycle
                return False # with cycle
        return True # no cycle

### Same idea but faster

In [ ]:
from collections import defaultdict

class Solution:
    def canFinish(self, numCourses: int, prerequisites: List[List[int]]) -> bool:
        # base cases
        if not numCourses:
            return False
        
        # pre-reqs are directed graphs
        # valid schedule must be a DAG
        
        courseDict = defaultdict(list)
        
        for x, y in prerequisites:
            courseDict[y].append(x) # white
          
        checked = [False] * numCourses # black
        path = [False] * numCourses # grey
        
        for currCourse in range(numCourses):
            if self.isCyclic(currCourse, courseDict, checked, path):
                return False
            
        return True
    
    def isCyclic(self, currCourse, courseDict, checked, path):
        # base case
        if checked[currCourse]:
            # this node's been fully visit, no cycle
            return False
        
        # check if we've seen this before
        if path[currCourse]:
            return True
        
        # mark as seen and do postorder DFS
        # append to grey
        path[currCourse] = True
        
        res = False
        
        # visit the children
        for child in courseDict[currCourse]:
            res = self.isCyclic(child, courseDict, checked, path)
            if res:
                break
                
        # after visiting the children, process the node
        path[currCourse] = False
        # append to black
        checked[currCourse] = True
        
        return res